# Web Scraping Tables using Pandas
========================================

**Estimated Effort: 5 mins**

The Pandas library in Python contains a function `read_html()` that can be used to extract tabular information from any web page.

This notebook covers:
- Using `pandas.read_html()` to extract tables
- Working with multiple tables on a webpage
- Handling limitations and data cleaning
- Practical examples with real websites

## Required Libraries

Let's start by importing the necessary libraries:

In [ ]:
# Install required packages if needed
# !pip install pandas lxml html5lib beautifulsoup4

import pandas as pd
import numpy as np
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")

## Example 1: Extracting Largest Banks Table

Let us assume we want to extract the list of the largest banks in the world by market capitalization, from the following link:

In [ ]:
# Define the URL for largest banks
URL = 'https://en.wikipedia.org/wiki/List_of_largest_banks'

print(f"Target URL: {URL}")
print("We will extract the table showing the largest banks by market capitalization.")

We may use `pandas.read_html()` function in python to extract all the tables in the web page directly.

**Note:** This is a live web page and it may get updated over time. The process of data extraction remains the same.

In [ ]:
# Extract all tables from the webpage
try:
    tables = pd.read_html(URL)
    print(f"Successfully extracted {len(tables)} tables from the webpage")
    
    # Display information about each table
    for i, table in enumerate(tables):
        print(f"Table {i}: Shape {table.shape} (rows, columns)")
        
except Exception as e:
    print(f"Error extracting tables: {e}")

In [ ]:
# Extract the first table (index 0) which contains the largest banks
df = tables[0]

print("Largest Banks Table:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\nFirst 10 rows:")
print(df.head(10))

In [ ]:
# Display basic information about the dataframe
print("DataFrame Info:")
print(df.info())
print("\n" + "="*50)

# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

## Example 2: Countries by GDP Table

Let's look at another example. Consider the following URL showing the list of countries by GDP (nominal):

In [ ]:
# Define the URL for countries by GDP
URL = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'

print(f"Target URL: {URL}")
print("This page contains multiple tables with GDP data.")

In [ ]:
# Extract all tables from the GDP webpage
try:
    tables = pd.read_html(URL)
    print(f"Successfully extracted {len(tables)} tables from the webpage")
    
    # Display detailed information about each table
    for i, table in enumerate(tables):
        print(f"\nTable {i}:")
        print(f"  Shape: {table.shape} (rows, columns)")
        print(f"  Columns: {list(table.columns)[:5]}...")  # Show first 5 columns
        
        # Show a sample of data
        if table.shape[0] > 0:
            print(f"  Sample data: {table.iloc[0, 0] if not pd.isna(table.iloc[0, 0]) else 'N/A'}")
            
except Exception as e:
    print(f"Error extracting tables: {e}")

**Important Note:** The contents of the tables in web pages may contain elements such as hyperlink text and other denoters, which are also scraped directly using the pandas method. This may lead to a requirement of further cleaning of data.

In [ ]:
# Extract the required table (index 2 in the original example)
# Note: Table indices may change as the webpage is updated
try:
    # Try different table indices to find the main GDP table
    for i in range(min(5, len(tables))):
        table = tables[i]
        if table.shape[0] > 10 and table.shape[1] >= 3:  # Look for substantial tables
            # Check if it contains country data
            first_col = str(table.iloc[0, 0]).lower()
            if any(keyword in first_col for keyword in ['country', 'rank', 'united', 'china']):
                print(f"Found main GDP table at index {i}")
                df_gdp = table
                break
    else:
        # Fallback to table 2 as in the original example
        df_gdp = tables[2]
        print("Using table index 2 (original example)")
    
    print(f"\nGDP Table Shape: {df_gdp.shape}")
    print(f"Columns: {list(df_gdp.columns)}")
    
except Exception as e:
    print(f"Error accessing GDP table: {e}")
    df_gdp = None

In [ ]:
# Display the GDP table
if df_gdp is not None:
    print("Countries by GDP (Nominal) - First 15 rows:")
    print(df_gdp.head(15))
    
    print("\n" + "="*60)
    print("Data Types:")
    print(df_gdp.dtypes)
else:
    print("GDP table not available")

## Limitations and Data Cleaning

Although convenient, the `pandas.read_html()` method comes with its own set of limitations:

1. **Hidden Tables**: Web pages may have content saved as tables but they may not appear as tables visually
2. **Extra Content**: Tables may contain hyperlinks, images, and other HTML elements
3. **Formatting Issues**: Data may need cleaning due to embedded HTML content

In [ ]:
# Example of data cleaning - removing unwanted characters
if df_gdp is not None and df_gdp.shape[0] > 0:
    # Create a copy for cleaning
    df_cleaned = df_gdp.copy()
    
    # Show original data
    print("Original data (first few rows):")
    print(df_gdp.head(3))
    
    # Clean string columns by removing common Wikipedia artifacts
    for col in df_cleaned.columns:
        if df_cleaned[col].dtype == 'object':
            # Remove common Wikipedia reference markers
            df_cleaned[col] = df_cleaned[col].astype(str).str.replace(r'\[.*?\]', '', regex=True)
            df_cleaned[col] = df_cleaned[col].str.replace(r'\(.*?\)', '', regex=True)
            df_cleaned[col] = df_cleaned[col].str.strip()
    
    print("\nCleaned data (first few rows):")
    print(df_cleaned.head(3))
    
    print("\nCleaning completed!")

## Advanced Usage Tips

Here are some additional tips for using `pandas.read_html()` effectively:

In [ ]:
# Advanced usage with parameters
def extract_tables_advanced(url, match=None, header=0):
    """
    Extract tables with advanced parameters
    
    Parameters:
    - url: Target URL
    - match: String or regex to match specific tables
    - header: Row to use as column headers
    """
    try:
        # Use additional parameters for better control
        tables = pd.read_html(
            url,
            match=match,           # Find tables containing specific text
            header=header,         # Specify header row
            skiprows=None,         # Skip rows if needed
            attrs=None,            # HTML attributes to match
            parse_dates=False,     # Parse date columns
            thousands=',',         # Thousands separator
            encoding=None,         # Character encoding
            decimal='.',           # Decimal separator
            converters=None,       # Column converters
            na_values=None,        # Additional NA values
            keep_default_na=True   # Keep default NA values
        )
        
        return tables
    
    except Exception as e:
        print(f"Error: {e}")
        return []

# Example usage
print("Advanced table extraction example:")

# Try to find tables containing "Bank" in the banks webpage
try:
    bank_tables = extract_tables_advanced(
        'https://en.wikipedia.org/wiki/List_of_largest_banks',
        match='Bank'
    )
    print(f"Found {len(bank_tables)} tables containing 'Bank'")
except:
    print("Advanced extraction failed, using basic method")

## Practical Exercise: Extract and Analyze Data

Let's practice with a complete workflow:

In [ ]:
# Complete workflow example
def scrape_and_analyze_table(url, table_index=0, save_csv=False):
    """
    Complete workflow for table scraping and basic analysis
    """
    try:
        # Step 1: Extract tables
        print(f"Extracting tables from: {url}")
        tables = pd.read_html(url)
        print(f"Found {len(tables)} tables")
        
        # Step 2: Select and examine table
        if table_index >= len(tables):
            print(f"Table index {table_index} not available. Using index 0.")
            table_index = 0
        
        df = tables[table_index]
        print(f"\nSelected table {table_index}: Shape {df.shape}")
        
        # Step 3: Basic cleaning
        df_clean = df.copy()
        
        # Clean string columns
        for col in df_clean.select_dtypes(include=['object']).columns:
            df_clean[col] = df_clean[col].astype(str).str.replace(r'\[.*?\]', '', regex=True)
            df_clean[col] = df_clean[col].str.strip()
        
        # Step 4: Basic analysis
        print(f"\nColumns: {list(df_clean.columns)}")
        print(f"Missing values: {df_clean.isnull().sum().sum()}")
        print(f"Duplicate rows: {df_clean.duplicated().sum()}")
        
        # Step 5: Display sample
        print("\nFirst 5 rows:")
        print(df_clean.head())
        
        # Step 6: Save if requested
        if save_csv:
            filename = f"scraped_table_{table_index}.csv"
            df_clean.to_csv(filename, index=False)
            print(f"\nTable saved as: {filename}")
        
        return df_clean
        
    except Exception as e:
        print(f"Error in workflow: {e}")
        return None

# Test the workflow
result = scrape_and_analyze_table(
    'https://en.wikipedia.org/wiki/List_of_largest_banks',
    table_index=0,
    save_csv=True
)

## Error Handling and Best Practices

When scraping tables, it's important to handle potential errors gracefully:

In [ ]:
# Robust table scraping function
def robust_table_scraper(url, max_retries=3, timeout=30):
    """
    Robust table scraping with error handling
    """
    import time
    
    for attempt in range(max_retries):
        try:
            print(f"Attempt {attempt + 1}/{max_retries}")
            
            # Add delay between retries
            if attempt > 0:
                time.sleep(2)
            
            # Try to read tables with timeout
            tables = pd.read_html(url, io=None)
            
            if tables:
                print(f"Success! Found {len(tables)} tables")
                return tables
            else:
                print("No tables found")
                
        except ImportError as e:
            print(f"Missing dependency: {e}")
            print("Try: pip install lxml html5lib beautifulsoup4")
            break
            
        except ValueError as e:
            print(f"No tables found: {e}")
            break
            
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            
    print("All attempts failed")
    return []

# Test robust scraper
print("Testing robust table scraper:")
robust_tables = robust_table_scraper('https://en.wikipedia.org/wiki/List_of_largest_banks')

if robust_tables:
    print(f"Successfully extracted {len(robust_tables)} tables")
else:
    print("Failed to extract tables")

## Summary

In this notebook, we learned how to:

1. **Extract tables** using `pandas.read_html()`
2. **Handle multiple tables** on a single webpage
3. **Clean scraped data** by removing HTML artifacts
4. **Use advanced parameters** for better control
5. **Implement error handling** for robust scraping
6. **Save results** to CSV files for further analysis

### Key Takeaways:

- `pandas.read_html()` is powerful but may need data cleaning
- Always inspect table structure before analysis
- Handle errors gracefully with try-except blocks
- Web pages change over time - code may need updates
- Consider using additional parameters for better results

### Next Steps:

- Learn about BeautifulSoup for more complex scraping
- Explore Selenium for dynamic content
- Practice with different websites and table structures
- Implement data validation and quality checks